In [1]:
# reuse the Image classification model
import torch
import os
import mlflow
import flash
from flash.core.data.utils import download_data
from flash.image import ImageClassificationData, ImageClassifier


/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/torchvision/models/_utils.py:252: UserWarning: Accessing the model URLs via the internal dictionary of the module is deprecated since 0.13 and will be removed in 0.15. Please access them via the appropriate Weights Enum instead.
  warnings.warn(


In [2]:
# Setup the mlflow experiment and AWS access
os.environ["MLFLOW_TRACKING_URI"] = "http://localhost"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"

In [3]:
datamodule = ImageClassificationData.from_folders(
    train_folder="../data-2/hymenoptera_data/train/",
    val_folder="../data-2/hymenoptera_data/val/",
    batch_size=2,
    transform_kwargs={"image_size": (196, 196), "mean": (0.485, 0.456, 0.406), "std": (0.229, 0.224, 0.225)},
)

# 2. Build the task
model = ImageClassifier(backbone="resnet18", labels=datamodule.labels)
trainer = flash.Trainer(max_epochs=1, gpus=torch.cuda.device_count())

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [4]:
# set experiment
EXPERIMENT_NAME = "dl_model"
mlflow.set_experiment(EXPERIMENT_NAME)
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
print("experiment_id:", experiment.experiment_id)

experiment_id: 1


In [5]:
mlflow.pytorch.autolog()

with mlflow.start_run(experiment_id=experiment.experiment_id, run_name="image-classifier") as dl_model_tracking_run:
    trainer.finetune(model, datamodule=datamodule, strategy="freeze")
    trainer.test()


  | Name          | Type           | Params
-------------------------------------------------
0 | train_metrics | ModuleDict     | 0     
1 | val_metrics   | ModuleDict     | 0     
2 | test_metrics  | ModuleDict     | 0     
3 | adapter       | DefaultAdapter | 11.2 M
-------------------------------------------------
10.6 K    Trainable params
11.2 M    Non-trainable params
11.2 M    Total params
44.710    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:322: UserWarning: The number of training samples (6) is smaller than the logging in

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/pytorch_lightning/trainer/callback_hook.py:102: LightningDeprecationWarning: The signature of `Callback.on_train_epoch_end` has changed in v1.3. `outputs` parameter has been removed. Support for the old signature will be removed in v1.5
  warning_cache.deprecation(
/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/pytorch_lightning/trainer/deprecated_api.py:25: LightningDeprecationWarning: `Trainer.running_sanity_check` has been renamed to `Trainer.sanity_checking` and will be removed in v1.5.
  rank_zero_deprecation(
/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:101: UserWarning: you defined a test_step but have no test_dataloader. Skipping test loop
  rank_zero_warn(f"you defined a {step_name} but have no {loader_name}. Skipping {stage} loop")
/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-pack

In [6]:
run_id = dl_model_tracking_run.info.run_id
print(run_id)
print(mlflow.get_run(run_id).info.lifecycle_stage)

98461adaab1d453c9f5573c49ce41193
active


In [7]:
# use the run_id to construct a logged_model URI.
logged_model = f'runs:/{run_id}/model'

In [8]:
# Load model 
loaded_model = mlflow.pytorch.load_model(logged_model)

In [ ]:
# predict
datamodule = ImageClassificationData.from_files(
    predict_files=[
        "../data-2/hymenoptera_data/predict/153783656_85f9c3ac70.jpg" ],
    batch_size=1,
)
predictions = trainer.predict(loaded_model, datamodule=datamodule)
print(predictions)

In [9]:
# register model
model_register = mlflow.register_model(logged_model, 'image-classifier')
print(model_register.name)
print(model_register.version)

Successfully registered model 'image-classifier'.
2022/10/18 10:36:43 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: image-classifier, version 1


image-classifier
1


Created version '1' of model 'image-classifier'.
